<a href="https://colab.research.google.com/github/barassah/Datasets/blob/main/Traditional_Models_AirlineDataset_Edited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Download NLTK stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Airline-Sentiment-2-w-AA.csv to Airline-Sentiment-2-w-AA.csv


In [ ]:
#reading and describing the dataset
dataset=pd.read_csv("Airline-Sentiment-2-w-AA.csv", encoding='ISO-8859-1')
dataset.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,airline_sentiment,airline_sentiment:confidence,negativereason,negativereason:confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,681448150,False,finalized,3,2/25/15 5:24,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2/24/15 11:35,5.703060e+17,NaN,Eastern Time (US & Canada)
1,681448153,False,finalized,3,2/25/15 1:53,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2/24/15 11:15,5.703010e+17,NaN,Pacific Time (US & Canada)
2,681448156,False,finalized,3,2/25/15 10:01,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2/24/15 11:15,5.703010e+17,Lets Play,Central Time (US & Canada)
3,681448158,False,finalized,3,2/25/15 3:05,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2/24/15 11:15,5.703010e+17,NaN,Pacific Time (US & Canada)
4,681448159,False,finalized,3,2/25/15 5:50,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2/24/15 11:14,5.703010e+17,NaN,Pacific Time (US & Canada)


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   _unit_id                      14640 non-null  int64  
 1   _golden                       14640 non-null  bool   
 2   _unit_state                   14640 non-null  object 
 3   _trusted_judgments            14640 non-null  int64  
 4   _last_judgment_at             14584 non-null  object 
 5   airline_sentiment             14640 non-null  object 
 6   airline_sentiment:confidence  14640 non-null  float64
 7   negativereason                9178 non-null   object 
 8   negativereason:confidence     10522 non-null  float64
 9   airline                       14640 non-null  object 
 10  airline_sentiment_gold        40 non-null     object 
 11  name                          14640 non-null  object 
 12  negativereason_gold           32 non-null     object 
 13  r

In [ ]:
dataset = dataset[['text']]  # Keep only text column
print(dataset)

                                                    text
0                    @VirginAmerica What @dhepburn said.
1      @VirginAmerica plus you've added commercials t...
2      @VirginAmerica I didn't today... Must mean I n...
3      @VirginAmerica it's really aggressive to blast...
4      @VirginAmerica and it's a really big bad thing...
...                                                  ...
14635  @AmericanAir thank you we got on a different f...
14636  @AmericanAir leaving over 20 minutes Late Flig...
14637  @AmericanAir Please bring American Airlines to...
14638  @AmericanAir you have my money, you change my ...
14639  @AmericanAir we have 8 ppl so we need 2 know h...

[14640 rows x 1 columns]


In [ ]:
# Text cleaning function
def clean_text(text):
    text = text.lower()
    # Modified regex to preserve words like "can't" and "don't"
    text = re.sub(r'[^\w\s]+', ' ', text) # remove punctuation but keep letters, numbers, and spaces
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    return text

# Apply text cleaning
dataset['text'] = dataset['text'].apply(clean_text)


In [ ]:
# Convert text into numerical features
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(dataset['text'])

In [ ]:
# Cluster into 2 groups (fake vs genuine)
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
pseudo_labels = kmeans.fit_predict(X)

# Add pseudo-labels to the dataset
dataset['label'] = pseudo_labels


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, pseudo_labels, test_size=0.3, random_state=42)


In [ ]:
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
svm_preds = svm.predict(X_test)
print("SVM Accuracy:", accuracy_score(y_test, svm_preds))


SVM Accuracy: 0.9929417122040073


In [ ]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
nb_preds = nb.predict(X_test)
print("Naïve Bayes Accuracy:", accuracy_score(y_test, nb_preds))


Naïve Bayes Accuracy: 0.8854735883424408


In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, rf_preds))


Random Forest Accuracy: 0.994535519125683


In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_preds = lr.predict(X_test)
print("Logistic Regression Accuracy:", accuracy_score(y_test, lr_preds))


Logistic Regression Accuracy: 0.9956739526411658


In [ ]:
# Print classification reports
print("SVM:\n", classification_report(y_test, svm_preds))
print("Naïve Bayes:\n", classification_report(y_test, nb_preds))
print("Random Forest:\n", classification_report(y_test, rf_preds))
print("Logistic Regression:\n", classification_report(y_test, lr_preds))


SVM:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00      3499
           1       0.97      1.00      0.98       893

    accuracy                           0.99      4392
   macro avg       0.98      1.00      0.99      4392
weighted avg       0.99      0.99      0.99      4392

Naïve Bayes:
               precision    recall  f1-score   support

           0       0.88      1.00      0.93      3499
           1       0.99      0.44      0.61       893

    accuracy                           0.89      4392
   macro avg       0.93      0.72      0.77      4392
weighted avg       0.90      0.89      0.87      4392

Random Forest:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00      3499
           1       0.98      1.00      0.99       893

    accuracy                           0.99      4392
   macro avg       0.99      1.00      0.99      4392
weighted avg       0.99      0.99     

In [ ]:
from sklearn.metrics import precision_recall_curve

precision_curve, recall_curve, _ = precision_recall_curve(true_labels, all_probs)

plt.plot(recall_curve, precision_curve, marker=".")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.show()

In [ ]:
#CNN
##import numpy as np
##import pandas as pd
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D, Dropout, Input
from tensorflow.keras.optimizers import Adam

# Load dataset (Modify path as needed)
#df = pd.read_csv("social_media_reviews.csv")  # Ensure columns: 'review' (text) and 'label' (0=fake,1=genuine)


In [ ]:
##texts = df["review"].astype(str).tolist()
##labels = df["label"].tolist()

# Encode labels
##label_encoder = LabelEncoder()
##labels = label_encoder.fit_transform(labels)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Apply TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=20000)  # Limit vocab size
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = tfidf_vectorizer.transform(X_test).toarray()

# Reshape for CNN (CNN expects 3D input: (samples, timesteps, features))
X_train_tfidf = np.expand_dims(X_train_tfidf, axis=-1)
X_test_tfidf = np.expand_dims(X_test_tfidf, axis=-1)

# CNN Model for Text Classification with TF-IDF
model = Sequential([
    Input(shape=(X_train_tfidf.shape[1], 1)),  # Input shape based on TF-IDF output
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification (fake vs genuine)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the CNN Model
BATCH_SIZE = 32
EPOCHS = 10

history = model.fit(X_train_tfidf, y_train, validation_data=(X_test_tfidf, y_test),
                    batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1)

# Evaluate on Test Data
test_loss, test_acc = model.evaluate(X_test_tfidf, y_test)
print(f"Test Accuracy: {test_acc:.4f}")
